In [191]:
#Libraries 
import pandas as pd
import json
from datetime import datetime
import string
from nltk.corpus import stopwords
from nltk import pos_tag
import nltk 

#Load Press Release file


# 'C:\Users\Ron\Desktop\Import into Python\Sales_’ + x1 + ‘.csv’        

with open('C:/Users/ADMIN/Downloads/Text_Stock/ISRG_pr.json', 'r',encoding='utf-8') as myfile:
    data=json.load(myfile)
df = pd.DataFrame(data)


newlist=[]
for i in df['date']:
    m="".join(str(i))
    m=m.lstrip()
    m=m.partition('AM')[0]
    m=m.partition('PM')[0]
    m=m.rsplit(' ', 1)[0]
    m=str(m)[0:]
    newdate=(datetime.strptime(m,'%b %d, %Y')) #%b %d,%Y
    newlist.append(newdate)


df1=pd.DataFrame(newlist)
df1.columns = ['date']
df1['date'].values
df['date']=df1['date']

#Load Stock File
df2=pd.read_csv('C:/Users/ADMIN/Downloads/Text_Stock/ISRG_STOCK.csv')
df['date']=df['date'].astype(str)
df['date']=pd.to_datetime(df['date'])
df2['date']=pd.to_datetime(df2['date'])


#df2['date'] = pd.to_datetime(df2['date'], format='%d/%m/%y').dt.strftime('%Y-%m-%d')

res=pd.merge(df,df2,on='date', how='outer')
res[' Close/Last']=res[' Close/Last'].str[1:]
res[' Close/Last']=res[' Close/Last'].replace({',': ''}, regex=True)
res[' Open']=res[' Open'].str[1:]
res[' Open']=res[' Open'].replace({',': ''}, regex=True)
res[' High']=res[' High'].str[1:]
res[' High']=res[' High'].replace({',': ''}, regex=True)
res[' Low']=res[' Low'].str[1:]
res[' Low']=res[' Low'].replace({',': ''}, regex=True)
res=res.dropna()
res[' Close/Last']=res[' Close/Last'].astype(float)
res[' Open']=res[' Open'].astype(float)
res[' High']=res[' High'].astype(float)
res[' Low']=res[' Low'].astype(float)

#New Column for DIfference of last and opening stock value 
res['diff']=res[' Close/Last']-res[' Open']



#Getting number of words in press release
res['number_of_words'] = res.content.apply(lambda x: len(x.split()))

#Remove Punctuation 
def remove_punctuation(text):
 no_punct = "".join([c for c in text if c not in string.punctuation])
 return no_punct
res['content']=res['content'].apply(lambda x:remove_punctuation(x))

#Getting POS tag 
tagged_item=res['content'].map(pos_tag)

#Getting count of POS tag
def count_tags(title_with_tags):
    tag_count = {}
    for word, tag in title_with_tags:
        if tag in tag_count:
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1
    return(tag_count)


bc1= pd.DataFrame() #Null Dataframe

#Dictionary for POS tag count 
x=tagged_item.map(count_tags)

#Converting Dictionary into Dataframe 
for row in x:
  bc=pd.DataFrame([row],columns=row.keys())
  bc1=bc1.append(bc)


res['pos']=res['diff']>0
bc1['Nature']=res['pos'].values
bc1['words']=res['number_of_words'].values


#Total positive and negative press release
Tr=(bc1['Nature']==True).sum()
Fl=(bc1['Nature']==False).sum()

#Total words in postiive and negative press release
Tr_cnt= bc1.loc[bc1['Nature']== True, 'words'].sum()
Fl_cnt= bc1.loc[bc1['Nature']== False,'words'].sum()

#Nomralization index
True_N=(Tr*Tr_cnt)
False_N=(Fl*Fl_cnt)


#Normalizing both the press releases 
sei=bc1.loc[bc1['Nature']==True]
sei1=(sei.loc[:, ~sei.columns.isin(['Nature', 'words'])])/True_N
sei1['Nat']=sei['Nature']

sei2=bc1.loc[bc1['Nature']==False]
sei3=(sei2.loc[:, ~sei2.columns.isin(['Nature', 'words'])])/False_N
sei3['Nat']=sei2['Nature']

sei4= pd.DataFrame(columns=['$','"','CC','CD','DT','FW','IN','JJ','MD','NN','NNP','NNS','PDT','PRP','RB','RP','SYM','VB','VBD','VBG','VBP','VBZ','WRB','Nat'])

sei5= pd.DataFrame(columns=['$','"','CC','CD','DT','FW','IN','JJ','MD','NN','NNP','NNS','PDT','PRP','RB','RP','SYM','VB','VBD','VBG','VBP','VBZ','WRB','Nat'])


sei4=sei4.append(sei1, ignore_index = True)
sei5=sei5.append(sei3,ignore_index = True)

#sei4.to_csv('C:/Users/ADMIN/Downloads/TAC/POS_new.csv', mode='a', header=False)
#sei5.to_csv('C:/Users/ADMIN/Downloads/TAC/Neg_new.csv', mode='a', header=False)

sei5




,"""",$,'',CC,CD,DT,FW,IN,JJ,MD,...,PRP,RB,RP,SYM,VB,VBD,VBG,VBP,VBZ,WRB
0,NaN,0.000761,0.000076,0.000228,0.011227,0.020894,0.000571,0.004339,0.030180,0.001560,...,0.000799,0.000609,NaN,0.000038,0.003425,0.005556,NaN,0.022606,0.012597,NaN
1,NaN,0.000495,0.000038,0.001789,0.008791,0.030789,0.000875,0.005556,0.046620,0.002207,...,0.002740,0.000571,NaN,0.000038,0.004643,0.006965,NaN,0.033605,0.020970,NaN
2,NaN,0.000723,0.000038,0.002093,0.035812,0.038362,0.000799,0.006584,0.056515,0.002930,...,0.003654,0.000761,NaN,0.000038,0.005899,0.015718,NaN,0.042548,0.025917,NaN
3,NaN,0.000076,0.000038,0.000152,0.001903,0.015870,0.000533,0.003806,0.022949,0.000228,...,0.000761,0.000343,NaN,NaN,0.000761,0.002892,NaN,0.016974,0.010884,NaN
